In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import quantile_transform
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import OneHotEncoder, LabelEncoder  ##. better to use dummy from pandas 
from sklearn.preprocessing import PowerTransformer, StandardScaler, minmax_scale
from scipy.stats import boxcox
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from scipy.stats import boxcox
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
pd.options.display.max_rows = 50
import random

In [ ]:
df = pd.read_csv("Data_Marketing_Customer_Analysis_Round3.csv")
df.info()
df.head()

In [ ]:
numbers = ["int64", ]
numericals = df.select_dtypes(include=numbers)
sns.heatmap(numericals.corr(), annot=True)

In [ ]:
#filter string features
categories = ["object"]
categoricals = df.select_dtypes(include=categories)
categoricals.drop(["effective_to_date"], axis=1, inplace=True)

In [ ]:
#split string features for label and one hot
label_features = ["coverage", "education", "vehicle_class","employment_status"]
label_cat = categoricals[label_features]
categoricals.drop(label_features, axis=1, inplace = True)

In [ ]:
label_cat

In [ ]:
#onehotencode strin features
encoder = OneHotEncoder()
hotcateg = encoder.fit_transform(categoricals).toarray()
hot_df = pd.DataFrame(hotcateg)
hot_df.columns = encoder.get_feature_names_out()
hot_df

In [ ]:
#labelencoding categorical features
labencoder = LabelEncoder()
lc_cov = labencoder.fit_transform(label_cat["coverage"])
lc_edu = labencoder.fit_transform(label_cat["education"])
lc_vc = labencoder.fit_transform(label_cat["vehicle_class"])
lc_es = labencoder.fit_transform(label_cat["employment_status"])

In [ ]:
#merging hot encoded features into one df
uni_df= np.stack([lc_cov, lc_edu, lc_vc, lc_es], axis=1)
labeled_df = pd.DataFrame(uni_df, columns=["coverage","education", "vehicle_class","employment_status"])
cat_trans_df = hot_df.join(labeled_df)
cat_trans_df

In [ ]:
#cat_dumms = pd.get_dummies(categoricals)

In [ ]:
numericals = numericals.join(cat_trans_df)

numericals

In [ ]:
#low_coef_feauteres = ["number_of_open_complaints","number_of_policies", "months_since_policy_inception","education", "months_since_last_claim"]
#numericals.drop(low_coef_feauteres, axis=1,inplace = True)

In [ ]:
pwr = PowerTransformer()
ltv = numericals["customer_lifetime_value"]
ltv_array = ltv.to_numpy()
ltv_array= np.reshape(ltv_array,(-1, 1))
ltv = pwr.fit_transform(ltv_array)
numericals["customer_lifetime_value"] = ltv


In [ ]:
stscl = StandardScaler()
mpa = numericals["income"]
mpa_array = mpa.to_numpy()
mpa_array= np.reshape(mpa_array,(-1, 1))
mpa = stscl.fit_transform(mpa_array)
numericals["income"] = mpa
numericals.hist(figsize=(14,14))

In [ ]:
#splitting the target feature
X = numericals.drop("total_claim_amount", axis=1)
y = numericals["total_claim_amount"]

In [ ]:
#splitting the testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state=123)
X_train = X_train.select_dtypes(include=np.number)
X_test  = X_test.select_dtypes(include=np.number)

**NEW WORK

In [ ]:
from sklearn.feature_selection import VarianceThreshold

#display(X_train)
print("Initial number of numerical columns: ",X_train.shape)
print()


selector = VarianceThreshold(threshold=100) # Default threshold value is 0
# Features with a training-set variance lower than this threshold will be removed.
selector.fit(X_train)

kept_features_indexes = selector.get_support(indices = True) #returns an array of integers corresponding to nonremoved features
kept_features = list(X_train.iloc[:,kept_features_indexes].columns)

X_train = selector.transform(X_train)
X_test  = selector.transform(X_test)

X_train = pd.DataFrame(X_train, columns=kept_features)
X_test  = pd.DataFrame(X_test, columns=kept_features)

print("Final number of numerical columns: ",X_train.shape)
print()
X_train

In [ ]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train=imp_mean.fit_transform(X_train)
X_test=imp_mean.fit_transform(X_test)

In [ ]:
pd.DataFrame(X_train)

In [ ]:
model=LinearRegression()
model.fit(X_train, y_train)
print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")

In [ ]:
###
from sklearn.linear_model import Lasso,Ridge,ElasticNet, LinearRegression
model=Lasso(alpha=0.05)

model.fit(X_train, y_train)
print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")

In [ ]:
model=Ridge(alpha=10000)
model.fit(X_train, y_train)
print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")

In [ ]:
model=ElasticNet(alpha=0.1)
model.fit(X_train, y_train)
print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
model=KNeighborsRegressor(n_neighbors=5)

model.fit(X_train, y_train)
print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")

**OLS

In [ ]:
X_added_constant = sm.add_constant(X)
X_added_constant

In [ ]:
# we need to add this constant value of 1 for the intercepts
model = sm.OLS(y,X_added_constant).fit()
model.summary()

In [ ]:
model.params[list(np.where(model.pvalues < 0.05)[0])].iloc[0:].index.tolist()
significant_features=X_added_constant[model.params[list(np.where(model.pvalues < 0.05)[0])].iloc[0:].index.tolist()]
significant_features

In [ ]:
model = sm.OLS(y,X_added_constant).fit()
model.summary()

**OLD WORK:

In [ ]:
std_scl = StandardScaler()
X_trained_tr = std_scl.fit_transform(X_train)
X_test_tr = std_scl.fit_transform(X_test)

In [ ]:
X_test_tr = std_scl.fit_transform(X_test)


In [ ]:
###statsmodels.api model
#train
X_trained_tr_cons = sm.add_constant(X_trained_tr)
model = sm.OLS(y_train, X_trained_tr_cons).fit()
predict_train = model.predict(X_trained_tr_cons)

#test
X_test_tr_cons = sm.add_constant(X_test_tr)
model = sm.OLS(y_test, X_test_tr_cons).fit()
predict_test = model.predict(X_test_tr_cons)

In [ ]:
print(model.summary())

In [ ]:
###Scikit model
sk_model = LinearRegression()
sk_model.fit(X_trained_tr, y_train)

In [ ]:
X_train

In [ ]:
len(sk_model.coef_)

In [ ]:
y_pred_test = sk_model.predict(X_test_tr)
y_pred_train = sk_model.predict(X_trained_tr)

In [ ]:
print(f'model score on training data: {sk_model.score(X_train, y_train)}')
print(f'model score on testing data: {sk_model.score(X_test, y_test)}')

In [ ]:
coefs = pd.DataFrame(
   sk_model.coef_,
   columns=['Coefficients'], index=X_train.columns)
coefs

In [ ]:
y_pred_train

In [ ]:
y_test

In [ ]:
#random prediciton sample
rps = []
for i in range(0, len(X_train.columns)):
    rps.append(X_train.iat[random.randrange(len(X_train)), i])
    

In [ ]:
len(rps)

In [ ]:
## new record prediction
std_scaler = StandardScaler()
new_data_scaled=std_scaler.fit_transform(np.array(rps).reshape(1,-1))

In [ ]:
np.array(np.array(rps).reshape(1,-1)).shape

In [ ]:
sk_model.predict(new_data_scaled)

In [ ]:
#checking prediction
result=pd.DataFrame({"y_test":y_test,"y_pred":y_pred_test})

In [ ]:
# Make an scatter plot y_pred vs y
# What kind of plot you will get if all the all the predictions are ok?
# A stright line

fig, ax = plt.subplots(1,3,figsize=(14,4))
ax[0].plot(y_pred_test, y_test, 'o')
ax[0].set_xlabel("y_test")
ax[0].set_ylabel("y_pred")
ax[0].set_title("Test Set -Predicted vs real")

# Get a histogram of the residuals ie: y - y_pred.  Homoscdasticity
# It resembles a normal distribution?
ax[1].hist(y_test - y_pred_test)
ax[1].set_xlabel("Test y-y_pred")
ax[1].set_title("Test Set Residual histogram")

ax[2].plot(y_pred_test,y_test - y_pred_test,"o")
ax[2].set_xlabel("predited")
ax[2].set_ylabel("residuals")
ax[2].set_title("Residuals by Predicted")
ax[2].plot(y_pred_test,np.zeros(len(y_pred_test)),linestyle='dashed')

In [ ]:
sns.regplot(x='y_pred',y='y_test', data=result, scatter_kws={"color": "red"}, line_kws={"color": "black"})

In [ ]:
print(mse(y_test,y_pred_test))
print(mae(y_test,y_pred_test))

In [ ]:
features_importances = pd.DataFrame(data={
    'Attribute': X_train.columns,
    'Importance': abs(sk_model.coef_)
})
features_importances = features_importances.sort_values(by='Importance', ascending=False)
features_importances

In [ ]:
df.to_csv("Kirill's_super_bad_prediction.csv")